# Voting Methods

**The code is included here as an example for you to refer to for your future projects.**

### Importing libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

RSEED=42

### Read CSV and splitting data

In [ ]:
# Import diabetes data
df = pd.read_csv('data/pima-native-americans-diabetes.csv', header=None)
df.head(2)

In [ ]:
# Define features and target and split into train and test set
y = df[8]
X = df.drop(8, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=RSEED)

In [ ]:
y_train.shape, y_test.shape

In [ ]:
y_train.value_counts(), y_test.value_counts()

## Max Voting 

In [ ]:
from sklearn.ensemble import VotingClassifier

model1 = LogisticRegression(random_state = RSEED)
model2 =  KNeighborsClassifier()
model3 = DecisionTreeClassifier(random_state = RSEED)

model = VotingClassifier(estimators = [('lr', model1), ('knn', model2), ('dt', model3)], voting = 'hard')
model.fit(X_train,y_train)
model.score(X_test,y_test)

## Averaging

In [ ]:
model1 = LogisticRegression(random_state = RSEED)
model2 = KNeighborsClassifier()
model3 = DecisionTreeClassifier(random_state = RSEED)

model1.fit(X_train,y_train)
model2.fit(X_train,y_train)
model3.fit(X_train,y_train)

pred1 = model1.predict_proba(X_test)
pred2 = model2.predict_proba(X_test)
pred3 = model3.predict_proba(X_test)

finalpred = (pred1 + pred2 + pred3) / 3
finalpred = np.argmax(finalpred.round(0), axis = 1)
(y_test == finalpred).sum() / len(finalpred)

## Weighted Average

In [ ]:
model1 = LogisticRegression(random_state = RSEED)
model2 = KNeighborsClassifier()
model3 = DecisionTreeClassifier(random_state = RSEED)

model1.fit(X_train,y_train)
model2.fit(X_train,y_train)
model3.fit(X_train,y_train)

pred1 = model1.predict_proba(X_test)
pred2 = model2.predict_proba(X_test)
pred3 = model3.predict_proba(X_test)

acc1 = accuracy_score(y_train, model1.predict(X_train))
acc2 = accuracy_score(y_train, model2.predict(X_train))
acc3 = accuracy_score(y_train, model3.predict(X_train))

acc_sum = acc1 + acc2 + acc3

weight1 = acc1/acc_sum
weight2 = acc2/acc_sum
weight3 = acc3/acc_sum

finalpred = (pred1*weight1 + pred2*weight2 + pred3*weight3)
finalpred = np.argmax(finalpred.round(0), axis = 1)
(y_test == finalpred).sum() / len(finalpred)

## Stacking 

In [ ]:
# Implementation of Stacking in Scikit-Learn
from sklearn.ensemble import StackingClassifier

estimators = [
    ('dt', DecisionTreeClassifier(random_state = RSEED)),
    ('knn', KNeighborsClassifier()),
    ('rf', RandomForestClassifier(random_state = RSEED))
]

clf = StackingClassifier(estimators = estimators, final_estimator = LogisticRegression())
clf.fit(X_train, y_train).score(X_test, y_test)

In order to simplify the example above, the stacking model we have created has only two levels. The **DecisionTree, KNN and RandomForest** models are built at **level zero**, while a **LogisticRegression** model is built at **level one**.

 

## Stacking Classifier explanation

### Stacking without `StackingClassifier`

We take 3 base classifiers and one final estimator separately. Initialize the object for each classifier

In [ ]:
# Base Estimators
dt = DecisionTreeClassifier(random_state = RSEED)
knn = KNeighborsClassifier()
rf = RandomForestClassifier(random_state = RSEED)

# final estimator
final_est = LogisticRegression()


For stacking classification we divide our train dataset into two parts
1. With the first part, we train our base estimators  
2. And with the second part we predict probabilities from base estimator and train the final estimator on the probabilities  

In [ ]:
X_train_1, X_train_2, y_train_1, y_train_2 = train_test_split(X_train, y_train, stratify=y_train, random_state=RSEED)

In [ ]:
# Fit all the base estimators on the 1st half of the train dataset
dt_model = dt.fit(X_train_1, y_train_1)
knn_model = knn.fit(X_train_1, y_train_1)
rf_model = rf.fit(X_train_1, y_train_1)

# Then with the second half of the train dataset we predict the probabilities from the base estimators
dt_probab = dt_model.predict_proba(X_train_2)[:,1]
knn_probab = knn_model.predict_proba(X_train_2)[:,1]
rf_probab = rf_model.predict_proba(X_train_2)[:,1]

In [ ]:
# Then we combine all the probabilities and form a training data (probabilities) for the final estimator
lr_X = pd.concat([
            pd.DataFrame(dt_probab), 
            pd.DataFrame(knn_probab),
            pd.DataFrame(rf_probab),
        ], axis=1)

In [ ]:
lr_X

In [ ]:
# Fit the final estimator on the combined probabilities and target values
final_est.fit(lr_X, y_train_2)

For the test dataset, we do the same thing as while training
1. predict probabilities from base estimators on the test dataset
2. combine the probabilities from base estimator to form a test dataset for final estimator
3. predict with final estimator on test data (probabilities)

In [ ]:
dt_pred = dt_model.predict_proba(X_test)[:,1]
knn_pred = knn_model.predict_proba(X_test)[:,1]
rf_pred = rf_model.predict_proba(X_test)[:,1]

comb_pred = pd.concat([
            pd.DataFrame(dt_pred), 
            pd.DataFrame(knn_pred),
            pd.DataFrame(rf_pred),
        ], axis=1)

pred_final = final_est.predict(comb_pred)

In [ ]:
from sklearn.metrics import f1_score, recall_score


print(accuracy_score(y_test, pred_final))
